In [1]:
import os
from softtek_llm.chatbot import Chatbot
from softtek_llm.models import OpenAI
from softtek_llm.cache import Cache
from softtek_llm.vectorStores import PineconeVectorStore
from softtek_llm.embeddings import OpenAIEmbeddings
from softtek_llm.schemas import Filter
from dotenv import load_dotenv

c:\Users\diego.anaya\OneDrive - Softtek\Desktop\SDKTest\.venv\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if OPENAI_API_KEY is None:
    raise ValueError("OPENAI_API_KEY not found in .env file")

OPENAI_API_BASE = os.getenv("OPENAI_API_BASE")
if OPENAI_API_BASE is None:
    raise ValueError("OPENAI_API_BASE not found in .env file")

OPENAI_EMBEDDINGS_MODEL_NAME = os.getenv("OPENAI_EMBEDDINGS_MODEL_NAME")
if OPENAI_EMBEDDINGS_MODEL_NAME is None:
    raise ValueError("OPENAI_EMBEDDINGS_MODEL_NAME not found in .env file")

OPENAI_CHAT_MODEL_NAME = os.getenv("OPENAI_CHAT_MODEL_NAME")
if OPENAI_CHAT_MODEL_NAME is None:
    raise ValueError("OPENAI_CHAT_MODEL_NAME not found in .env file")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
if PINECONE_API_KEY is None:
    raise ValueError("PINECONE_API_KEY not found in .env file")

PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT")
if PINECONE_ENVIRONMENT is None:
    raise ValueError("PINECONE_ENVIRONMENT not found in .env file")

PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
if PINECONE_INDEX_NAME is None:
    raise ValueError("PINECONE_INDEX_NAME not found in .env file")

In [3]:
vector_store = PineconeVectorStore(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRONMENT,
    index_name=PINECONE_INDEX_NAME,
)
embeddings_model = OpenAIEmbeddings(
    api_key=OPENAI_API_KEY,
    model_name=OPENAI_EMBEDDINGS_MODEL_NAME,
    api_type="azure",
    api_base=OPENAI_API_BASE,
)
cache = Cache(
    vector_store=vector_store,
    embeddings_model=embeddings_model,
)
model = OpenAI(
    api_key=OPENAI_API_KEY,
    model_name=OPENAI_CHAT_MODEL_NAME,
    api_type="azure",
    api_base=OPENAI_API_BASE,
    verbose=True,
)
filters = [
    Filter(
        type="DENY",
        case="ANYTHING about the Titanic. YOU CANNOT talk about the Titanic AT ALL.",
    )
]
chatbot = Chatbot(
    model=model,
    description="You are a very helpful and polite chatbot",
    filters=filters,
    cache=cache,
    verbose=True,
)

In [4]:
response = chatbot.chat(
    "Hello! My name is Jeff.",
    print_cache_score=True,
    cache_kwargs={"namespace": "chatbot-test"},
)

Revision prompt: Considering this context:


Please review the prompt below and answer with "yes" if it adheres to the rules or "no" if it violates any of the rules.
Rules:
	- DENY: ANYTHING about the Titanic. YOU CANNOT talk about the Titanic AT ALL.

Prompt:
Hello! My name is Jeff.
Memory: [{'role': 'system', 'content': "As a prompt reviser, your role is to determine whether the given prompt respects the rules provided below. You can only respond with 'yes' or 'no.' The rules are absolute, meaning that if any of them are not respected, the prompt is considered invalid."}, {'role': 'system', 'content': "only respond with 'yes' or 'no'"}, {'role': 'user', 'content': 'Considering this context:\n\n\nPlease review the prompt below and answer with "yes" if it adheres to the rules or "no" if it violates any of the rules.\nRules:\n\t- DENY: ANYTHING about the Titanic. YOU CANNOT talk about the Titanic AT ALL.\n\nPrompt:\nHello! My name is Jeff.'}]
Memory: [{'role': 'system', 'content': 'You 

In [5]:
response

Response(message=Message(role='assistant', content="Hello Jeff! It's nice to meet you. How can I assist you today?"), created=1695402445, latency=781, from_cache=False, model='gpt-35-turbo-16k', usage=Usage(prompt_tokens=27, completion_tokens=17, total_tokens=44), additional_kwargs={})

In [6]:
response = chatbot.chat(
    "What is my name?",
    print_cache_score=True,
    cache_kwargs={"namespace": "chatbot-test"},
)

Revision prompt: Considering this context:
	- user: Hello! My name is Jeff.
	- assistant: Hello Jeff! It's nice to meet you. How can I assist you today?

Please review the prompt below and answer with "yes" if it adheres to the rules or "no" if it violates any of the rules.
Rules:
	- DENY: ANYTHING about the Titanic. YOU CANNOT talk about the Titanic AT ALL.

Prompt:
What is my name?
Memory: [{'role': 'system', 'content': "As a prompt reviser, your role is to determine whether the given prompt respects the rules provided below. You can only respond with 'yes' or 'no.' The rules are absolute, meaning that if any of them are not respected, the prompt is considered invalid."}, {'role': 'system', 'content': "only respond with 'yes' or 'no'"}, {'role': 'user', 'content': 'Considering this context:\n\t- user: Hello! My name is Jeff.\n\t- assistant: Hello Jeff! It\'s nice to meet you. How can I assist you today?\n\nPlease review the prompt below and answer with "yes" if it adheres to the rule

In [7]:
response

Response(message=Message(role='assistant', content='Your name is Jeff, as you mentioned earlier. Is there anything else I can help you with, Jeff?'), created=1695402446, latency=1031, from_cache=False, model='gpt-35-turbo-16k', usage=Usage(prompt_tokens=57, completion_tokens=22, total_tokens=79), additional_kwargs={})

In [8]:
response = chatbot.chat(
    "When did the Titanic sink?",
    print_cache_score=True,
    cache_kwargs={"namespace": "chatbot-test"},
)

Revision prompt: Considering this context:
	- assistant: Hello Jeff! It's nice to meet you. How can I assist you today?
	- user: What is my name?
	- assistant: Your name is Jeff, as you mentioned earlier. Is there anything else I can help you with, Jeff?

Please review the prompt below and answer with "yes" if it adheres to the rules or "no" if it violates any of the rules.
Rules:
	- DENY: ANYTHING about the Titanic. YOU CANNOT talk about the Titanic AT ALL.

Prompt:
When did the Titanic sink?
Memory: [{'role': 'system', 'content': "As a prompt reviser, your role is to determine whether the given prompt respects the rules provided below. You can only respond with 'yes' or 'no.' The rules are absolute, meaning that if any of them are not respected, the prompt is considered invalid."}, {'role': 'system', 'content': "only respond with 'yes' or 'no'"}, {'role': 'user', 'content': 'Considering this context:\n\t- assistant: Hello Jeff! It\'s nice to meet you. How can I assist you today?\n\t-

InvalidPrompt: The prompt does not follow the rules set by the filters. If this behavior is not intended, consider modifying the filters. It is recommended to use LLMs for meta prompts.

In [ ]:
chatbot.cache.vector_store.delete(delete_all=True, namespace="chatbot-test")